# OpenStreetMap Data Case Study

## Map Area

New York, NY, United States

- http://www.openstreetmap.org/relation/175905

- https://mapzen.com/data/metro-extracts/metro/new-york_new-york/

I'm Korean and I live in South Korea. And I've never been to US before. I already heard about New York and Manhatten many times, and if I can get a opportunity I want to visit there. Therefore I'm interested to do the OpenStreetMap data case study with New York part of the map and see the results of querying.

## Problems in the map that I decided to solve

After making a small sample size of the New York area and creating database with the sample, I noticed lots of problems in the data. In this section I'm going to show the problems that I decided to solve and discuss here.

- Inconsistent postal codes (“NY 10533”, “08854-8009”, “10314”)


- Incorrect postal codes (New York area zip codes begin with “005”or "063" or "1". However some zip codes start with "07" or "08". And there are even some zip codes which aren't 5 digit number nor zip+4 code.)


- Same value type but different key names("postcode" and "postal_code", "fax" and "Fax")


- Inconsistent phone, fax numbers("(718) 623-9065", "+1 212 690 4000", "718-733-6813", etc)


- Inconsistent website addresses("bk.com", "http://www.nycgovparks.org/parks/Q048/", "www.skyviewonthehudson.com/")

### Postal codes

In [6]:
#Connect to osm_samples.db using sqlite3.
import sqlite3

db = sqlite3.connect("C:\\sqlite_windows\\osm_samples.db")
c = db.cursor()

https://www.unitedstateszipcodes.org/ny/

### Same value type but different key names

### Inconsistent phone and fax numbers

### Inconsistent website addresses

## How did I solve above problems

### Postal codes

In [ ]:
if key in ['postcode', 'postal_code']:
    if key == 'postal_code':
        key = 'postcode'   
    if value[:2].isdigit() == False:
        value = value[2:].strip()
    if (len(value) == 5 and value.isdigit()) == False and \
        (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
            continue

- First, I changed the key name 'postal_code' to 'postcode' for consistency. And then, if the first two digits are not numbers like 'NY', I removed them and extra spaces. After that, I only accepted the value whose format is 5 digit number or the 10 figures of zip+4 code.

### Inconsistent phone and fax numbers

In [ ]:
if key in ['phone', 'fax', 'Fax']:
    if key == 'Fax':
        key = 'fax'
    if re.search(r'[a-z|A-Z]', value):
        continue
    if value[:2] in ['+1', '01', '1 ', '1-']:
        value = value[2:]
    for ch in [' ', '-', '.', '(', ')']:
        if ch in value:
            value = value.replace(ch, '')
    if (len(value) == 10 and value.isdigit()) == False:
        continue
    value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])

- First, I changed the key name 'Fax' to 'fax' for consistency. And then, I removed the key and value if there is any alphabet in the value using re module and 'continue'. If the first two figures are '+1' or '01' or '1 ' or '1-', I removed the part to make '(xxx) xxx - xxxx' format. And I temporarily removed ' ', '-', '(', ')' to check whether the remained part of the value is 10 digit number. If not, I removed the key and value again. Finally, I changed the value to make it follow the phone number format.

### Inconsistent website addresses

In [ ]:
if key == 'website':
    if value[:4] != 'http':
        value = 'http://' + value

- If the value doesn't start with 'http', I added 'http://'.

### Same value type but different key names

- Before entered into cleaning process of postcode and fax number, I added "**if key == 'A': key = 'B'**" sentence to change key name 'A' to 'B'.

## Additional problems in the map

### Maxspeed

In [11]:
uncleaned_maxspeed_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'maxspeed' and tags.type = 'regular' \
        order by tags.value;"

('10',)
('10 mph',)
('110 mph',)
('125 mph',)
('15',)
('15 mph',)
('160',)
('177',)
('20 mph',)
('201',)
('217',)
('25 mph',)
('30',)
('30 mph',)
('35',)
('35 mph',)
('40 mph',)
('45 mph',)
('48',)
('50',)
('50 mph',)
('55 mph',)
('60',)
('60 mph',)
('64',)
('65 mph',)
('70 mph',)
('72',)
('75 mph',)
('80',)
('90 mph',)
('Amtrak ETT 2010-01-18',)
('Conrail ETT 2013-07-01',)
('Conrail ETT 2013-10-17',)
('signals',)

- Some of them are just number and others are number added by 'mph'. According to the [wiki](http://wiki.openstreetmap.org/wiki/Key:maxspeed), 'km/h(or kph)' is the basic unit of maxspeed. And the value which not measured by kph should be added to the end of it. 


- But when I see the numbers which aren't added by any unit, some of them are finished by '5' or '0', and others are finished by '1', '2', '7', etc. I think that it is reasonable to think that the former numbers' unit is 'mph', the [US standard speed unit](https://en.wikipedia.org/wiki/Miles_per_hour), and the latter numbers are converted to 'km/h' from 'mph'(because 48, 64, 72, 177, 201, 217 km/h can be converted to 30, 40, 45, 110, 125, 135 mph each). But it is my guess and to correctly add units, I need to find the maxspeed information for each place. Approximate speed limits in New York is in [here](https://en.wikipedia.org/wiki/Speed_limits_in_the_United_States_by_jurisdiction#New_York).


- And some values are about maxspeed's source, not about speed limit. They entered into here because when I process the keys which have lower colons in them, I divided them by ':' and the left part became types and the right part became keys. Therefore in this case of database, I need to use "where tags.key = 'maxspeed' and tags.type = 'regular'" in the query to extract only speed limit information.

### Opening_hours

In [22]:
uncleaned_oh_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'opening_hours' and tags.value not like '%__:__%' \
        order by tags.value;"

('24/7',)
('5pm - 9pm (weekdays) , 2pm - 8pm ( weekend)',)
('772',)
('Mon - Fri, 6am - 9pm; Sat, 6am - 8am; Sun, 6am - 7pm',)
('open until 4am',)

- There are some rules to write opening hours data in the [wiki](http://wiki.openstreetmap.org/wiki/Key:opening_hours). But we can find that there are some datas which don't follow the rules. In the results, some of them use 'am' or 'pm' instead of 'hh:mm' syntax. Also, some of them use words like 'until' and 'to' when it is possible to use '-' symbol. It can be a small problem when users read the datas, but it can be a big problem to classify them programmatically. It is a consistency matter.

### Same value type but different key names

In [24]:
uncleaned_cusine_query = "select * \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'cusine';"

(4528128545, 'cusine', 'thai', 'regular')

- It is also the similar key name problem but in this time it looked like that the similar key name was made by mistake. Most of cuisines (112 of them) are in the key named 'cuisine' and there is a single cuisine in the key named 'cusine'.

## Ideas for improving the data

## Data overview and additional data exploration

### File sizes

In [5]:
import os
osm_path = 'C:\\projects\\New_York_OSM\\New_York_sample.osm'
db_path = 'C:\\sqlite_windows\\osm_samples.db'
print('New_York_sample.osm :', os.stat(osm_path).st_size)
print('osm_samples.db :', os.stat(db_path).st_size)

basedir = 'C:\\projects\\New_York_OSM\\samples'
names = os.listdir(basedir)
paths = [os.path.join(basedir, name) for name in names[:5]]
sizes = [os.stat(path).st_size for path in paths]
for i in range(5):
    print(names[i], ':', sizes[i])

New_York_sample.osm : 96696280
osm_samples.db : 107073536
sample_nodes.csv : 34343067
sample_nodes_tags.csv : 960657
sample_ways.csv : 3927976
sample_ways_nodes.csv : 11669712
sample_ways_tags.csv : 9811433


### I'm going to explore about building, sport, and shop fields. And I'm going to query at nodes_tags and ways_tags tables. 

### Top 5 usage of buildings.

In [2]:
building_query = "select tags.value, count(*) as num \
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'building' and tags.value != 'yes' \
        group by tags.value \
        order by num desc \
        limit 5;"

('house', 5393)
('garage', 4088)
('shed', 489)
('commercial', 317)
('service', 218)

- Most of the buildings are for house or garage in New York.

### Top 5 sports facilities.

In [3]:
sport_query = "select tags.value, count(*) as num \
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'sport' and tags.value != 'yes' \
        group by tags.value \
        order by num desc \
        limit 5;"

('baseball', 67)
('tennis', 65)
('basketball', 35)
('soccer', 25)
('american_football', 11)

- 5 of the most common sports facilities in New York are for baseball, tennis, basketball, soccer and american football.

### The number of shop information.

In [4]:
shop_query = "select tags.key, count(*) as num\
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'shop';"

('shop', 211)

### The number of shop information which have phone number or email or website or facebook.

In [5]:
shop_with_phone_query = "select tags.key, count(*) as num\
        from (select * from nodes_tags union all select * from ways_tags) tags \
        join \
        (select distinct(id) from nodes_tags where key in ('phone', 'email', 'website', 'facebook') \
            union all \
         select distinct(id) from ways_tags where key in ('phone', 'email', 'website', 'facebook')) ids \
        on tags.id = ids.id \
        where tags.key = 'shop';"

('shop', 40)

- The number of shop information in New York OSM sample is 211 but only 40 of them have contact information. I think that it is better to make each shop information to have at least one contact information in the map.

## Conclusion